In [1]:
import logging
import os
import copy
import math
import pathlib
from datetime import datetime
import csv

# 3rd Party
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
from PIL import Image

# Pytorch
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
from torch.autograd import Variable
import torchvision
import torchvision.transforms as transforms


In [2]:
# Japanese compatible font
plt.rcParams['font.sans-serif'] = "Microsoft YaHei" 

# Computation device setup
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = torch.device('cpu')
if torch.cuda.is_available():
    print(f"Using device: {torch.cuda.get_device_name(0)}")
    os.environ['CUDA_LAUNCH_BLOCKING'] = "0"
else:
    print(f"Using device: CPU")

Using device: NVIDIA GeForce GTX 1060 6GB


# Utils

In [3]:
def plot_images(images):
    plt.figure(figsize=(32, 32))
    plt.imshow(torch.cat([
        torch.cat([i for i in images.cpu()], dim=-1),
    ], dim=-2).permute(1, 2, 0).cpu())
    plt.show()

def save_images(images, path, **kwargs):
    grid = torchvision.utils.make_grid(images, nrow=5, **kwargs)
    ndarr = grid.permute(1, 2, 0).to('cpu').numpy()
    im = Image.fromarray(ndarr)
    im.save(path)

## Dataset

In [4]:
DATASET_DIR = 'hiragana'
DATASET_TRAIN_FILE = 'hiragana.csv'

In [5]:
train_labels = []
train_images = []
# validate_labels = []
# validate_images = []
with open(os.path.join(DATASET_DIR, DATASET_TRAIN_FILE), 'r') as csv_file:
    csvreader = csv.reader(csv_file)
    i = 0
    for data in csvreader:
        train_labels.append(int(data[0]))
        train_images.append([int(a) for a in data[1:]])
        # if i%20==0:
        #     validate_labels.append(int(data[0]))
        #     validate_images.append([int(a) for a in data[1:]])
        # else:
        #     train_labels.append(int(data[0]))
        #     train_images.append([int(a) for a in data[1:]])
        # i += 1
train_images = torch.tensor(train_images, dtype=torch.float32).reshape(-1,32,32).to(device)
train_labels = torch.tensor(train_labels, dtype=torch.int64).to(device)
# validate_images = torch.tensor(validate_images, dtype=torch.float32).reshape(-1,32,32).to(device)
# validate_labels = torch.tensor(validate_labels, dtype=torch.int64).to(device)
print(f"Training Set: Input shape: {train_images.shape}. Output shape: {train_labels.shape}")
# print(f"Validating Set: Input shape: {validate_images.shape}. Output shape: {validate_labels.shape}")

Training Set: Input shape: torch.Size([920, 32, 32]). Output shape: torch.Size([920])


In [6]:
# Add one dimension for channel
train_images = train_images.unsqueeze(1)
print(f"Training Set: Input shape: {train_images.shape}")
# validate_images = validate_images.unsqueeze(1)
# print(f"Validating Set: Input shape: {validate_images.shape}")

Training Set: Input shape: torch.Size([920, 1, 32, 32])


In [7]:
# shift pixel values into [-1,1]
def shiftInputImage(images):
    return (images-128)/128
train_images = shiftInputImage(train_images)
# validate_images = shiftInputImage(validate_images)

In [8]:
torch.cuda.empty_cache() 

In [9]:
# CLASS_NUM = torch.unique(train_labels).shape[0]
# print(f"Label class number: {CLASS_NUM}")
# print(torch.unique(train_labels))

# assert torch.unique(validate_labels).shape[0] == CLASS_NUM
CLASS_NUM = 49

# Model

In [10]:
class Diffusion:
    def __init__(self, noise_step=1000, beta_start=1e-4, beta_end=0.02, img_size=64):
        self.noise_step = noise_step
        self.beta_start = beta_start
        self.beta_end = beta_end
        self.img_size = img_size

        self.beta = self.prepare_noise_schedule().to(device)
        self.alpha = 1 - self.beta
        self.alpha_hat = torch.cumprod(self.alpha, dim=0)

    def prepare_noise_schedule(self):
        return torch.linspace(self.beta_start, self.beta_end, self.noise_step)

    def noise_images(self, x, t):
        sqrt_alpha_hat = torch.sqrt(self.alpha_hat[t]).reshape(t.shape[0], 1, 1, 1).repeat(1, 1, self.img_size, self.img_size)
        sqrt_one_minus_alpha_hat = torch.sqrt(1 - self.alpha_hat[t]).reshape(t.shape[0], 1, 1, 1).repeat(1, 1, self.img_size, self.img_size)
        Ɛ = torch.randn_like(x)
        return (
            (torch.mul(sqrt_alpha_hat, x)+torch.mul(sqrt_one_minus_alpha_hat,Ɛ)),
            Ɛ
        )

    def sample_timesteps(self, n):
        return torch.randint(low=1, high=self.noise_step, size=(n,))

    # denoise process
    def sample(self, model, n, labels, cfg_scale=3):
        logging.info(f"Sampling {n} new images ...")
        model.eval()
        with torch.no_grad():
            x = torch.randn((n, 1, self.img_size, self.img_size)).to(device)
            # for i in tqdm( reversed(range(1, self.noise_step)), position=0):
            for i in reversed(range(1, self.noise_step)):
                t_tensor = (torch.ones(n) * i).long().to(device)
                t = i
                predicted_noise = model(x, t_tensor, labels)
                if cfg_scale > 0:
                    uncond_predicted_noise = model(x, t_tensor, None)
                    predicted_noise = torch.lerp(uncond_predicted_noise, predicted_noise, cfg_scale)
                alpha = self.alpha[t]
                alpha_hat = self.alpha_hat[t]
                beta = self.beta[t]
                if i > 1:
                    noise = torch.randn_like(x)
                else:
                    # in the final iteration, we don't want to add noise back to X0
                    noise = torch.zeros_like(x)
                x = 1 / torch.sqrt(alpha) * (x - ((1 - alpha) / (torch.sqrt(1 - alpha_hat))) * predicted_noise) + torch.sqrt(beta) * noise

        model.train()
        x = (x.clamp(-1, 1) + 1) / 2
        x = (x * 255).type(torch.uint8)
        return x

In [11]:
class SelfAttention(nn.Module):
    def __init__(self, channels, size):
        super(SelfAttention, self).__init__()
        self.channels = channels
        self.size = size
        self.mha = nn.MultiheadAttention(channels, 4, batch_first=True)
        self.ln = nn.LayerNorm([channels])
        self.ff_self = nn.Sequential(
            nn.LayerNorm([channels]),
            nn.Linear(channels, channels),
            nn.GELU(),
            nn.Linear(channels, channels),
        )

    def forward(self, x):
        x = x.view(-1, self.channels, self.size * self.size).swapaxes(1, 2)
        x_ln = self.ln(x)
        attention_value, _ = self.mha(x_ln, x_ln, x_ln)
        attention_value = attention_value + x
        attention_value = self.ff_self(attention_value) + attention_value
        return attention_value.swapaxes(2, 1).view(-1, self.channels, self.size, self.size)


class DoubleConv(nn.Module):
    def __init__(self, in_channels, out_channels, mid_channels=None, residual=False):
        super().__init__()
        self.residual = residual
        if not mid_channels:
            mid_channels = out_channels
        self.double_conv = nn.Sequential(
            nn.Conv2d(in_channels, mid_channels, kernel_size=3, padding=1, bias=False),
            nn.GroupNorm(1, mid_channels),
            nn.GELU(),
            nn.Conv2d(mid_channels, out_channels, kernel_size=3, padding=1, bias=False),
            nn.GroupNorm(1, out_channels),
        )

    def forward(self, x):
        if self.residual:
            return F.gelu(x + self.double_conv(x))
        else:
            return self.double_conv(x)


class Down(nn.Module):
    def __init__(self, in_channels, out_channels, emb_dim=256):
        super().__init__()
        self.maxpool_conv = nn.Sequential(
            nn.MaxPool2d(2),
            DoubleConv(in_channels, in_channels, residual=True),
            DoubleConv(in_channels, out_channels),
        )

        self.emb_layer = nn.Sequential(
            nn.SiLU(),
            nn.Linear(
                emb_dim,
                out_channels
            ),
        )

    def forward(self, x, t):
        x = self.maxpool_conv(x)
        emb = self.emb_layer(t)[:, :, None, None].repeat(1, 1, x.shape[-2], x.shape[-1])
        return x + emb


class Up(nn.Module):
    def __init__(self, in_channels, out_channels, emb_dim=256):
        super().__init__()

        self.up = nn.Upsample(scale_factor=2, mode="bilinear", align_corners=True)
        self.conv = nn.Sequential(
            DoubleConv(in_channels, in_channels, residual=True),
            DoubleConv(in_channels, out_channels, in_channels // 2),
        )

        self.emb_layer = nn.Sequential(
            nn.SiLU(),
            nn.Linear(
                emb_dim,
                out_channels
            ),
        )

    def forward(self, x, skip_x, t):
        x = self.up(x)
        x = torch.cat([skip_x, x], dim=1)
        x = self.conv(x)
        emb = self.emb_layer(t)[:, :, None, None].repeat(1, 1, x.shape[-2], x.shape[-1])
        return x + emb
    
class UNet_Conditional(nn.Module):
    # sf: scaling factor of cnn kernel numbers
    def __init__(self, c_in=1, c_out=1, time_dim=256, num_classes=None, sf=16):
        super().__init__()
        self.time_dim = time_dim
        self.inc = DoubleConv(c_in, sf*1)
        self.down1 = Down(sf*1, sf*2)
        self.sa1 = SelfAttention(sf*2, 16)
        self.down2 = Down(sf*2, sf*4)
        self.sa2 = SelfAttention(sf*4, 8)
        self.down3 = Down(sf*4, sf*4)
        self.sa3 = SelfAttention(sf*4, 4)

        self.bot = DoubleConv(sf*4, sf*4)

        self.up1 = Up(sf*4+sf*4, sf*2)
        self.sa4 = SelfAttention(sf*2, 8)
        self.up2 = Up(sf*2+sf*2, sf*1)
        self.sa5 = SelfAttention(sf*1, 16)
        self.up3 = Up(sf*1+sf*1, sf*1)
        self.sa6 = SelfAttention(sf*1, 32)
        self.outc = nn.Conv2d(sf*1, c_out, kernel_size=1)

        if num_classes is not None:
            self.label_emb = nn.Embedding(num_classes, time_dim)

    def pos_encoding(self, t, channels):
        inv_freq = 1.0 / (
            10000
            ** (torch.arange(0, channels, 2, device=device).float() / channels)
        )
        pos_enc_a = torch.sin(t.repeat(1, channels // 2) * inv_freq)
        pos_enc_b = torch.cos(t.repeat(1, channels // 2) * inv_freq)
        pos_enc = torch.cat([pos_enc_a, pos_enc_b], dim=-1)
        return pos_enc

    # y: labels (the condition of diffusion)
    def forward(self, x, t, y):
        t = t.unsqueeze(-1).type(torch.float)
        t = self.pos_encoding(t, self.time_dim)
        if y is not None:
            t += self.label_emb(y)

        x1 = self.inc(x)
        x2 = self.down1(x1, t)
        x2 = self.sa1(x2)
        x3 = self.down2(x2, t)
        x3 = self.sa2(x3)
        x4 = self.down3(x3, t)
        x4 = self.sa3(x4)

        x4 = self.bot(x4)

        x = self.up1(x4, x3, t)
        x = self.sa4(x)
        x = self.up2(x, x2, t)
        x = self.sa5(x)
        x = self.up3(x, x1, t)
        x = self.sa6(x)
        output = self.outc(x)
        return output

In [12]:
class EMA:
    def __init__(self, beta):
        super().__init__()
        self.beta = beta
        self.step = 0

    def update_model_average(self, ma_model, current_model):
        for current_params, ma_params in zip(current_model.parameters(), ma_model.parameters()):
            old_weight, up_weight = ma_params.data, current_params.data
            ma_params.data = self.update_average(old_weight, up_weight)

    def update_average(self, old, new):
        if old is None:
            return new
        return old * self.beta + (1 - self.beta) * new

    def step_ema(self, ema_model, model, step_start_ema=2000):
        if self.step < step_start_ema:
            self.reset_parameters(ema_model, model)
            self.step += 1
            return
        self.update_model_average(ema_model, model)
        self.step += 1

    def reset_parameters(self, ema_model, model):
        ema_model.load_state_dict(model.state_dict())

# Train

In [13]:
def train(epochs=500, batch_size=6, image_size=32, lr=3e-4, timestep=200, name=None):
    model = UNet_Conditional(c_in=1, c_out=1, num_classes=CLASS_NUM, sf=8).to(device)
    optimizer = optim.AdamW(model.parameters(), lr=lr)
    mse = nn.MSELoss()
    diffusion = Diffusion(img_size=image_size, noise_step=200)

    ema = EMA(0.95)
    ema_model = copy.deepcopy(model).eval().requires_grad_(False)

    if name is None:
        name = datetime.now().strftime("%Y%m%d_%H%M")
    
    pathlib.Path(f'results/{name}').mkdir(parents=True, exist_ok=True) 

    train_total_step = math.ceil(train_images.shape[0] / batch_size)

    prev_val_loss = math.inf
    for epoch in range(epochs):
        print(f"Starting epoch {epoch}:")
        train_error = 0
        model.train()
        print(f"Train:")
        pbar = tqdm(range(train_total_step))
        for i in pbar:
            optimizer.zero_grad()
            if ((i+1)*batch_size > train_images.shape[0]):
                images = train_images[i*batch_size:]
                labels = train_labels[i*batch_size:]
            else:
                images = train_images[i*batch_size:(i+1)*batch_size]
                labels = train_labels[i*batch_size:(i+1)*batch_size]
            t = diffusion.sample_timesteps(images.shape[0]).to(device)
            x_t, noise = diffusion.noise_images(images, t)
            if np.random.random() < 0.05: ## ?
                labels = None
            predicted_noise = model(x_t, t, labels)
            loss = mse(noise, predicted_noise)
            train_error += loss.item()
            loss.backward()
            optimizer.step()
            ema.step_ema(ema_model, model)

            pbar.set_postfix(MSE=(train_error/(i+1)))

            if (epoch * train_total_step + i)%400 == 0:
                labels = torch.unique(train_labels).long().to(device)[:25]
                torch.cuda.empty_cache()
                sampled_images = diffusion.sample(model, n=len(labels), labels=labels)
                save_images(sampled_images, os.path.join("results", f"{name}", f"{(epoch * train_total_step + i)}.jpg"))
                torch.cuda.empty_cache()
                ema_sampled_images = diffusion.sample(ema_model, n=len(labels), labels=labels)
                save_images(ema_sampled_images, os.path.join("results", f"{name}", f"EMA_{(epoch * train_total_step + i)}.jpg"))
        avg_train_error = (train_error/train_total_step)

        # Save model
        if (epoch+1)%100 == 0:
            torch.save(model.state_dict(), os.path.join("results", f"{name}", f"model_epoch_{epoch+1}.pt"))
                
        torch.cuda.empty_cache()

In [14]:
#train(epochs=5000, batch_size=72, image_size=32, lr=5e-5)
# with EMA
# timestep = 200
# lowest MSE=0.045

In [15]:
# timestep = 500
# train(epochs=1000, batch_size=72, image_size=32, lr=5e-5, timestep=500)
# lowest MSE=0.055
# 20240322_2017

In [16]:
train(epochs=1500, batch_size=72, image_size=32, lr=5e-5, timestep=50, name="conditional_ema_t50")

Starting epoch 0:
Train:


100%|████████████████████████████████████████████████████████████████████████| 13/13 [00:29<00:00,  2.27s/it, MSE=1.03]


Starting epoch 1:
Train:


100%|███████████████████████████████████████████████████████████████████████| 13/13 [00:02<00:00,  4.54it/s, MSE=0.935]


Starting epoch 2:
Train:


...

100%|██████████████████████████████████████████████████████████████████████| 13/13 [00:03<00:00,  4.14it/s, MSE=0.0483]


Starting epoch 1498:
Train:


100%|███████████████████████████████████████████████████████████████████████| 13/13 [00:03<00:00,  4.19it/s, MSE=0.048]


Starting epoch 1499:
Train:


100%|██████████████████████████████████████████████████████████████████████| 13/13 [00:03<00:00,  4.23it/s, MSE=0.0498]


In [17]:
torch.cuda.empty_cache()
train(epochs=1500, batch_size=72, image_size=32, lr=5e-5, timestep=5, name="conditional_ema_t5")

Starting epoch 0:
Train:


100%|████████████████████████████████████████████████████████████████████████| 13/13 [00:29<00:00,  2.25s/it, MSE=1.36]


Starting epoch 1:
Train:


100%|████████████████████████████████████████████████████████████████████████| 13/13 [00:03<00:00,  4.23it/s, MSE=1.14]


Starting epoch 2:
Train:


...

100%|██████████████████████████████████████████████████████████████████████| 13/13 [00:03<00:00,  4.08it/s, MSE=0.0519]


Starting epoch 1498:
Train:


100%|██████████████████████████████████████████████████████████████████████| 13/13 [00:03<00:00,  4.11it/s, MSE=0.0517]


Starting epoch 1499:
Train:


100%|██████████████████████████████████████████████████████████████████████| 13/13 [00:03<00:00,  4.08it/s, MSE=0.0515]


## TODO
- why EMA doesn't work? is there outlier?
- cosine scheduler
- higher time step